<a href="https://colab.research.google.com/github/IgnasiOliveras/anonimitzar/blob/main/MERGED_FINAL_DEF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd

# Cargar el archivo principal
df_principal_subset = pd.read_excel('/content/30.01.25 Uic.xlsx', sheet_name='Export')

# Filtrar solo ContactId 466 y 300
#df_principal_subset = df_principal[df_principal['ContactId'].isin([466, 300])]

# Convertir a datetime
df_principal_subset["fecha"] = pd.to_datetime(df_principal_subset["createdAt"], format="%d/%m/%Y").dt.date

# Calcular rango de 10 segundos
start_time = df_principal_subset["createdAt"].dt.floor('10S')
end_time = start_time + pd.Timedelta(seconds=10)

df_principal_subset["rango"] = (
    start_time.dt.strftime("%H:%M:%S") + "-" + end_time.dt.strftime("%H:%M:%S")
)

# Cargar el archivo de fichas CRM
df_fichas_crm_subset = pd.read_excel('/content/20.02.25FichasCrm (2).xlsx')

# Filtrar solo ContactId 466 y 300
#df_fichas_crm_subset = df_fichas_crm[df_fichas_crm['ContactId'].isin([466, 300])]

# Convertir fechas y renombrar columna ContactId
df_fichas_crm_subset["fecha"] = pd.to_datetime(df_fichas_crm_subset["Data/hora inici"], errors="coerce", dayfirst=True)
df_fichas_crm_subset["fecha_str"] = df_fichas_crm_subset["fecha"].dt.strftime("%d/%m/%Y")
df_fichas_crm_subset = df_fichas_crm_subset.rename(columns={'Número de telèfon de l’usuari': 'ContactId2'})

# Calcular rango de 10 segundos
start_time = df_fichas_crm_subset["fecha"].dt.floor('10S')
end_time = start_time + pd.Timedelta(seconds=10)

df_fichas_crm_subset["rango2"] = (
    start_time.dt.strftime("%H:%M:%S") + "-" + end_time.dt.strftime("%H:%M:%S")
)

# Cargar el archivo de tipificaciones
df_tipificaciones_subset = pd.read_excel('/content/20.02.25TipificacionesConversaciones.xlsx')

# Filtrar solo ContactId 466 y 300
#df_tipificaciones_subset = df_tipificaciones[df_tipificaciones['ContactId'].isin([466, 300])]

# Renombrar columnas
df_tipificaciones_subset = df_tipificaciones_subset.rename(columns={'ContactId': 'ContactId1'})

# Realizar el primer LEFT JOIN entre df_principal_subset y df_tipificaciones_subset
df_merged = pd.merge(
    df_principal_subset,
    df_tipificaciones_subset,
    how='left',
    left_on=['OpenchannelInteractionId', 'ContactId'],
    right_on=['id', 'ContactId1']
)


df_merged2 = pd.merge(
    df_principal_subset,
    df_fichas_crm_subset,
    how='left',
    left_on=['ContactId', 'fecha'],
    right_on=[ 'ContactId', 'fecha_str']

)


df_merged2 = df_merged2.drop_duplicates()


df_merged.to_excel('df_merged.xlsx', index=False)
df_merged2.to_excel('df_merged2.xlsx', index=False)




<ipython-input-4-a7bb1340462f>:13: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  start_time = df_principal_subset["createdAt"].dt.floor('10S')
<ipython-input-4-a7bb1340462f>:32: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  start_time = df_fichas_crm_subset["fecha"].dt.floor('10S')


In [6]:

# Remove duplicate IDs, keeping the first occurrence by default
df_merged2 = df_merged2.drop_duplicates(subset=['id'], keep='first')

print(df_merged2.head())

df_merged2.to_excel('df_merged2.xlsx', index=False)

      id                                               body  read  secret  \
0  77623                                               Hola     1       0   
1  77624  Benvingut/da a *Xat de suport emocional per a ...     1       0   
2  77625                                   Me llamo sakhina     1       0   
3  77626                                              Hola!     1       0   
4  77627  Antes de empezar a chatear, es necesario que l...     1       0   

  direction              readAt  providerName  providerResponse  \
0        in 2023-01-01 00:11:06           NaN               NaN   
1       out                 NaT           NaN               NaN   
2        in 2023-01-01 00:11:07           NaN               NaN   
3       out                 NaT           NaN               NaN   
4       out                 NaT           NaN               NaN   

            createdAt           updatedAt  ...  \
0 2023-01-01 00:10:19 2023-01-01 00:11:06  ...   
1 2023-01-01 00:10:19 2023-01-01 0

In [29]:

import pandas as pd

# Cargar los archivos
df_merged = pd.read_excel("/content/df_merged.xlsx")
df_merged2 = pd.read_excel("/content/df_merged2.xlsx")

# --- Asegurar formatos consistentes ---
# Convertir todas las fechas a datetime
# Convertir todas las fechas a datetime
df_merged["fecha"] = pd.to_datetime(df_merged["fecha"])
# Changed line: specify the correct format for 'fecha_str'
df_merged2["fecha_str"] = pd.to_datetime(df_merged2["fecha_str"], format="%d/%m/%Y")

# Ensure 'rango2' column is of string type before applying string methods
df_merged2["rango2"] = df_merged2["rango2"].astype(str)

# Normalizar rangos de tiempo (ej: "14:30:20-14:30:30")
df_merged2["rango2"] = df_merged2["rango2"].str.replace(" ", "").str.strip()


# --- Realizar el JOIN con columnas alineadas ---
# Changed lines: Rename 'rango' in df_merged and 'rango2' in df_merged2 to avoid duplicate column names
df_final = pd.merge(
    df_merged.rename(columns={"rango": "rango_merged"}),
    df_merged2.rename(columns={
        "fecha_str": "fecha",  # Renombrar columna para igualar nombres
        "rango2": "rango_merged2"
    }),
    left_on=['fecha',"rango_merged", "ContactId"],
    right_on=['fecha',"rango_merged2", "ContactId"],  # Ahora las columnas tienen los mismos nombres
    how="left"
)

# Eliminar las primeras 20 columnas
df_final = df_final.iloc[:, 20:]
# Eliminar filas duplicadas
df_final = df_final.drop_duplicates()

# Guardar el archivo final
df_final.to_excel("/content/df_final_limpio.xlsx", index=False)

# Mostrar información del resultado
print("Se han eliminado las primeras 20 columnas y los duplicados.")
print("Filas en df_final después de limpieza:", len(df_final))
print(df_final.head())

Se han eliminado las primeras 20 columnas y los duplicados.
Filas en df_final después de limpieza: 745
   id_y  closed            closedAt     disposition note  ContactId1       id  \
0  1489       1 2023-01-01 00:35:18  Conversa atesa  NaN         466  77623.0   
1  1489       1 2023-01-01 00:35:18  Conversa atesa  NaN         466  77624.0   
2  1489       1 2023-01-01 00:35:18  Conversa atesa  NaN         466  77625.0   
3  1489       1 2023-01-01 00:35:18  Conversa atesa  NaN         466  77626.0   
4  1489       1 2023-01-01 00:35:18  Conversa atesa  NaN         466  77627.0   

                                              body_y  read_y  secret_y  ...  \
0                                               Hola     1.0       0.0  ...   
1  Benvingut/da a *Xat de suport emocional per a ...     1.0       0.0  ...   
2                                   Me llamo sakhina     1.0       0.0  ...   
3                                              Hola!     1.0       0.0  ...   
4  Antes de emp

In [31]:
import pandas as pd

# Cargar los archivos
df_merged = pd.read_excel("/content/df_merged.xlsx")
df_merged2 = pd.read_excel("/content/df_merged2.xlsx")

# --- Asegurar formatos consistentes ---
# Convertir todas las fechas a datetime
df_merged["fecha"] = pd.to_datetime(df_merged["fecha"])
df_merged2["fecha_str"] = pd.to_datetime(df_merged2["fecha_str"], format="%d/%m/%Y")

# Asegurar que 'rango2' sea string antes de aplicar métodos de cadena
df_merged2["rango2"] = df_merged2["rango2"].astype(str).str.replace(" ", "").str.strip()

# --- Realizar el JOIN solo por 'ContactId' ---
df_final = pd.merge(df_merged, df_merged2, on="ContactId", how="left")

# Eliminar las primeras 20 columnas
df_final = df_final.iloc[:, 20:]

# Eliminar filas duplicadas
df_final = df_final.drop_duplicates(subset=['id'], keep='first')

# Guardar el archivo final
df_final.to_excel("/content/df_final_limpio.xlsx", index=False)

# Mostrar información del resultado
print("Se ha realizado el join solo por 'ContactId'.")
print("Se han eliminado las primeras 20 columnas y los duplicados.")
print("Filas en df_final después de limpieza:", len(df_final))
print(df_final.head())


Se ha realizado el join solo por 'ContactId'.
Se han eliminado las primeras 20 columnas y los duplicados.
Filas en df_final después de limpieza: 6943
   id_y  closed            closedAt     disposition note  ContactId1     id  \
0  1489       1 2023-01-01 00:35:18  Conversa atesa  NaN         466  77623   
1  1489       1 2023-01-01 00:35:18  Conversa atesa  NaN         466  77624   
2  1489       1 2023-01-01 00:35:18  Conversa atesa  NaN         466  77625   
3  1489       1 2023-01-01 00:35:18  Conversa atesa  NaN         466  77626   
4  1489       1 2023-01-01 00:35:18  Conversa atesa  NaN         466  77627   

                                              body_y  read_y  secret_y  ...  \
0                                               Hola       1         0  ...   
1  Benvingut/da a *Xat de suport emocional per a ...       1         0  ...   
2                                   Me llamo sakhina       1         0  ...   
3                                              Hola!       